# KNN算法原理

给定一个训练集,对新输入的实例,在训练集中找到与该实例最近邻的k个实例,这k个实例的多数属于某个类,我们就把输入实例分为这个类.

## KNN算法描述

输入:训练集$T={(x_1,y_1),(x_2,y_2),...,(x_N,y_N)}$,其中$x_i\in X$ 为实例的特征向量,$y_i\in {c_1,c_2,...,c_m}$为实例的类别;实例特征向量x

输出:实例x所属的类别y
1. 根据给定的距离度量方式,在训练集T中找到与x最邻近的k个点,涵盖着k个点的x的邻域记作$N_k(x)$
2. 在$N_k(x)$中根据分类决策规则决定x的类别y
$$y = argmax_{c_j}\sum_{(x_i\in N_k(x))}I(y_i = c_j)$$
其中$I(y_i = c_j)$为指示函数,当$y_i = c_j$的时候I = 1,后者I = 0

## KNN的基本要素
对于一个确定的训练集,只要确定了**距离度量** **k值**和**分类决策规则**,就能对任何一个新的实例,确定它的分类
### 距离度量
距离度量是描述特征空间中两个实例的距离,也是这两个实例的相似程度.在n维实数向量空间中,我们主要使用的距离是欧氏距离,但也可以使用更加一般化的$L_p$距离(闵可夫斯基距离).

在特征空间中,取出两个特征$x_i, x_j$,它们分别是n维的特征向量.
1. 欧氏距离 $$L_2(x_i,x_j) = (\sum^n_{l=1}|x_i^l-x_j^l|)^\frac{1}{2}$$
2. 曼哈顿距离 $$L_1(x_i,x_j) = sum^n_{l=1}|x_i^l-x_j^l|$$
3. 闵可夫斯基距离 $$L_p(x_i,x_j) = (\sum^n_{l=1}|x_i^l-x_j^l|^\frac{1}{p}$$

### k值的选择
对于k值的选择没有一个固定的经验,一般按照样本的分布,选择一个较小的值,然后通过**交叉验证选择一个合适的k值**
* 选择较小的k值，就相当于用较小的领域中的训练实例进行预测，训练误差会减小，只有与输入实例较近或相似的训练实例才会对预测结果起作用，与此同时带来的问题是泛化误差会增大。换句话说，**k值的减小就意味着整体模型变得复杂，容易发生过拟合**。
* 选择较大的k值，就相当于用较大领域中的训练实例进行预测，其优点是可以减少泛化误差，但缺点是训练误差会增大。这时候，与输入实例较远（不相似的）训练实例也会对预测器作用，使预测发生错误。换句话说，**k值的增大就意味着整体的模型变得简单，容易发生欠拟合**。

## kd树
KNN算法最简单的实现方式，就是好计算输入实例和所有训练实例的距离，然后进行排序，取前k个，进行分类。但是训练集特别大的时候，这种方式非常耗时，不可行。下面介绍kd树的方式，kd树是通过减少输入实例和训练实例的计算次数来达到优化的目的.

kd树算法包括三步,第一步是建树,第二步是搜索最近邻,最后一步是预测
### 构造kd树
kd树是一种对n维空间的实例点进行存储,以便对其进行快速检索的树形结构.kd树是二叉树,构造kd树相当于不断用垂直于坐标轴的超平面 对n维空间进行
划分,构成一系列的n维超矩阵区域.

kd树建树采用的是从m个样本的n维特征中,分别计算n个特征的取值的方差,用方差最大的第k维特征$n_k$来作为根节点.对于这个特征,我们选择$n_k$取值的中位数$n_{kv}$对应的样本作为划分点,对于所有第k维特征的取值小于$n_{kv}$的样本,我们划入左子树,对于第k维特征的取值大于等于$n_{kv}$的样本,我们划入右子树,对于左子树和右子树,我们采用和刚才同样的办法来找方差最大的特征来做更节点,递归的生成kd树.
### kd树搜索最近邻
当生成kd树以后,就可以预测测试集里的样本目标点了.预测的过程如下: 
1. 从根节点开始,递归的往下移.往左还是往右的决定方法与插入元素的方法一样(如果输入点在分区的左边则进入左子节点,在右边则进入右子节点).
2. 一旦移动到叶节点,将该节点当作"当前最佳点".
3. 解开递归,并对每个经过的节点运行下列步骤:  
1)如果当前所在点比当前最佳点更靠近输入点,则将其变为当前最佳点.  
2)检查另一边子树有没有更近的点,如果有则从该节点往下找.    
4. 当根节点搜索完毕后完成最邻近搜索.

## 优缺点
* 优点:精度高 对异常值不敏感 无数据输入假定
* 缺点:计算复杂度高 空间复杂度高
适用数据范围:数值型和标称型

# KNN算法实现

## sklearn库中实现k近邻算法
* 分类问题：from sklearn.neighbors import KNeighborsClassifier
* 回归问题：from sklearn.neighbors import KNeighborsRegressor

1.数据蓝蝴蝶

In [1]:
#导包
import numpy as np
import pandas  as pd
from pandas import DataFrame,Series
from sklearn.neighbors import KNeighborsClassifier #k邻近算法模型

#使用datasets创建数据
import sklearn.datasets as datasets
iris = datasets.load_iris()

feature = iris['data']
target = iris['target']

#将样本打乱，符合真实情况

np.random.seed(1)
np.random.shuffle(feature)
np.random.seed(1)
np.random.shuffle(target)

#训练数据
x_train = feature[:140]
y_train = target[:140]
#测试数据
x_test = feature[-10:]
y_test =target[-10:]

#实例化模型对象&训练模型
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train,y_train)
knn.score(x_train,y_train)

print('预测分类：',knn.predict(x_test))
print('真实分类：',y_test)

预测分类： [0 2 1 2 1 2 2 1 2 0]
真实分类： [0 2 1 2 1 2 2 1 2 0]


2.根据身高 体重 鞋子尺码,预测性别

In [2]:
#导包
import numpy as np
import pandas  as pd
from pandas import DataFrame,Series

#手动创建训练数据集
feature = np.array([[170,65,41],[166,55,38],[177,80,39],[179,80,43],[170,60,40],[170,60,38]])
target = np.array(['男','女','女','男','女','女'])

from sklearn.neighbors import KNeighborsClassifier #k邻近算法模型

#实例k邻近模型，指定k值=3
knn = KNeighborsClassifier(n_neighbors=3)

#训练数据
knn.fit(feature,target)

#模型评分
knn.score(feature,target)

#预测
knn.predict(np.array([[176,71,38]]))

array(['男'], dtype='<U1')